In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# 環境変数の読み込み
load_dotenv("../.env")
os.environ['OPENAI_API_KEY'] = os.environ['API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages

# ステートの定義
class State(TypedDict):
    # データを保存する属性
    messages: Annotated[list, add_messages]

# ステートグラフの作成
graph_builder = StateGraph(State)

In [3]:
# 言語モデルの定義
llm = ChatOpenAI(model_name=MODEL_NAME)

# チャットボットノードの作成
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# グラフにチャットボットノードを追加
graph_builder.add_node("chatbot", chatbot)

# 開始ノードの指定
graph_builder.set_entry_point("chatbot")
# 終了ノードの指定
graph_builder.set_finish_point("chatbot")

# 実行可能なステートグラフの作成
graph = graph_builder.compile()

In [4]:
# グラフの実行
response = graph.invoke({"messages": [("user", "光の三原色は？")]})

# 結果の表示
print(response)

{'messages': [HumanMessage(content='光の三原色は？', additional_kwargs={}, response_metadata={}, id='b3f410a6-af39-4527-b89c-4d53918c221d'), AIMessage(content='光の三原色は、赤（Red）、緑（Green）、青（Blue）です。これらの色を組み合わせることで、さまざまな色を作り出すことができます。この概念は、RGBカラーシステムに基づいており、ディスプレイや照明などの技術で広く利用されています。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 14, 'total_tokens': 95, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'finish_reason': 'stop', 'logprobs': None}, id='run-a307c170-c791-4b42-83a2-737faddff7bc-0', usage_metadata={'input_tokens': 14, 'output_tokens': 81, 'total_tokens': 95, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}


In [5]:
# 言語モデルからの回答のみを表示
print(response["messages"][-1].content)

光の三原色は、赤（Red）、緑（Green）、青（Blue）です。これらの色を組み合わせることで、さまざまな色を作り出すことができます。この概念は、RGBカラーシステムに基づいており、ディスプレイや照明などの技術で広く利用されています。


In [7]:
# グラフの実行と結果の表示
def stream_graph_updates(user_input: str):
    # 結果をストリーミングで得る
    events = graph.stream({"messages": [("user", user_input)]})
    for event in events:
        for value in event.values():
            print("回答:", value["messages"][-1].content, flush=True)

# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    print("質問:", user_input, flush=True)
    stream_graph_updates(user_input)

質問: こんにちは！
回答: こんにちは！どういったことをお話ししましょうか？
質問: aで始まる英単語を5つ教えて
回答: もちろんです！以下は「a」で始まる英単語の例です：

1. Apple（リンゴ）
2. Adventure（冒険）
3. Artist（アーティスト）
4. Amazing（素晴らしい）
5. Animal（動物）

何か他に知りたいことがあれば教えてください！
質問: 3つ目の英単語は何ですか？
回答: 「3つ目の英単語」というのは具体的な文脈がないと特定できませんが、何か特定のテーマやリストがある場合は教えていただければ、そのテーマに関連する3つ目の英単語をお答えできます。どういったことに関してお尋ねでしょうか？
ありがとうございました!


In [8]:
from langgraph.checkpoint.memory import MemorySaver

# チェックポインタの作成
memory = MemorySaver()

# 記憶を持つ実行可能なステートグラフの作成
memory_graph = graph_builder.compile(checkpointer=memory)


In [9]:
# グラフの実行と結果の表示
def stream_graph_updates(user_input: str):
    events = memory_graph.stream(
        {"messages": [("user", user_input)]},
        {"configurable": {"thread_id": "1"}},
        stream_mode="values")
    # 結果をストリーミングで得る
    for event in events:
        print(event["messages"][-1].content, flush=True)

# チャットボットのループ
while True:
    user_input = input("質問:")
    if user_input.strip()=="":
        print("ありがとうございました!")
        break
    stream_graph_updates(user_input)

こんにちは！
こんにちは！どういったことにお手伝いできますか？
aで始まる英単語を5つ教えて
もちろんです！以下は「a」で始まる英単語の例です。

1. Apple（リンゴ）
2. Amazing（驚くべき）
3. Adventure（冒険）
4. Artist（アーティスト）
5. Animal（動物）

他にも知りたい単語があれば教えてください！
4つ目の英単語は何ですか？
4つ目の英単語は「Artist」（アーティスト）です。アーティストは、芸術家や創作活動を行う人を指します。何か他に知りたいことがあれば教えてください！
ありがとうございました!
